**import libraries**

In [3]:
import pandas as pd

### **Read Dataset**

In [2]:
!gdown --id 1XulBQYyiPRXMpwfm-RIvaDLCLDKIU6UD

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:127: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1XulBQYyiPRXMpwfm-RIvaDLCLDKIU6UD
To: /content/review_10_product_women.csv
100% 9.48M/9.48M [00:00<00:00, 10.1MB/s]


In [14]:
data = pd.read_csv('/content/review_10_product_women.csv')

In [15]:
data.head()

,stars,title,body
0,4.0 out of 5 stars,Short people and big booty Judy’s.,Honest review would be more like a 4.5. Issue ...
1,4.0 out of 5 stars,Great Pants,I LOVE these pants! I originally bought 2x bec...
2,5.0 out of 5 stars,Kinda big,I usually wear a medium however this was close...
3,5.0 out of 5 stars,OMG I love these pants.,They feel so good and smooth. Great if you are...
4,4.0 out of 5 stars,"Nice, but large","These are comfortable, but as others have ment..."


In [16]:
data.shape

(49802, 3)

In [17]:
data['star'] = data['stars'].apply(lambda x: str(x).split()[0]).astype(float)

In [18]:
data.head(3)

,stars,title,body,star
0,4.0 out of 5 stars,Short people and big booty Judy’s.,Honest review would be more like a 4.5. Issue ...,4.0
1,4.0 out of 5 stars,Great Pants,I LOVE these pants! I originally bought 2x bec...,4.0
2,5.0 out of 5 stars,Kinda big,I usually wear a medium however this was close...,5.0


In [19]:
data['sentiment'] = data['star'].apply(lambda x: 1 if x >= 3 else 0)

In [20]:
data.head(3)

,stars,title,body,star,sentiment
0,4.0 out of 5 stars,Short people and big booty Judy’s.,Honest review would be more like a 4.5. Issue ...,4.0,1
1,4.0 out of 5 stars,Great Pants,I LOVE these pants! I originally bought 2x bec...,4.0,1
2,5.0 out of 5 stars,Kinda big,I usually wear a medium however this was close...,5.0,1


In [21]:
data['sentiment'].unique()

array([1, 0])

In [22]:
data['sentiment'].value_counts()

1    45180
0     4622
Name: sentiment, dtype: int64